# Exploring Ray API Calls

This lesson explores a few of the other API calls you might find useful, as well as options that can be used with the API calls we've already learned. If you are pressed for time, consider skipping this lesson now and proceeding to the recap lesson [06-RecapTipsTricks](06-RecapAndTipsTricks.ipynb).

> **Tip:** The [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [2]:
# If you are running on Google Colab, uncomment and run the following linkes
# to install the necessary dependencies.

# print("Setting up colab environment")
# !pip install -q ray

In [3]:
import ray, time, sys

In [4]:
def pnd(n, duration, prefix=''):
    """Print an integer and a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}n: {:2d}, duration: {:6.3f} seconds'.format(prefix2, n, duration))

def pd(duration, prefix=''):
    """Print a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}duration: {:6.3f} seconds'.format(prefix2, duration))

In [6]:
ray.init(ignore_reinit_error=True)

2020-04-16 13:53:54,447	INFO resource_spec.py:212 -- Starting Ray with 4.93 GiB memory available for workers and up to 2.47 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-16 13:53:54,846	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:64881',
 'object_store_address': '/tmp/ray/session_2020-04-16_13-53-54_434638_40563/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-16_13-53-54_434638_40563/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-16_13-53-54_434638_40563'}

## ray.init()

When we used [`ray.init()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init), we used it to start Ray on our local machine. When the optional `address=...` argument is specified, the driver connects to the corresponding Ray cluster.

There are a lot of optional keyword arguments you can pass to `ray.init()`. Here are some of them. All options are described in the [documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.init).

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `address` | `str` | `address='auto'` | The address of the Ray cluster to connect to. If this address is not provided, then this command will start Redis, a raylet, a plasma store, a plasma manager, and some workers. It will also kill these processes when Python exits. If the driver is running on a node in a Ray cluster, using `auto` as the value tells the driver to detect the the cluster, removing the need to specify a specific node address. |
| `num_cpus` | `int` | `num_cpus=4` | Number of CPUs the user wishes to assign to each _raylet_. |
| `num_gpus` | `int` | `num_gpus=1` | Number of GPUs the user wishes to assign to each _raylet_. |
| `resources` | `dictionary` | `resources={'resource1': 4, 'resource2': 16}` | Maps the names of custom resources to the quantities of those resources available. |
| `memory` | `int` | `memory=1000000000` | The amount of memory (in bytes) that is available for use by workers requesting memory resources. By default, this is automatically set based on the available system memory. |
| `object_store_memory` | `int` | `object_store_memory=1000000000` | The amount of memory (in bytes) for the object store. By default, this is automatically set based on available system memory, subject to a 20GB cap. |
| `log_to_driver` | `bool` | `log_to_driver=True` | If true, then the output from all of the worker processes on all nodes will be directed to the driver program. |
| `local_mode` | `bool` | `local_mode=True` | If true, the code will be executed serially. This is useful for debugging. |
| `ignore_reinit_error` | `bool` | `ignore_reinit_error=True` | If true, Ray suppresses errors from calling `ray.init()` a second time (as we've done in these notebooks). Ray won't be restarted. |
| `include_webui` | `bool` | `include_webui=False` | Boolean flag indicating whether or not to start the web UI, which displays the status of the Ray cluster. By default, or if this argument is `None`, then the UI will be started if the relevant dependencies are present. |
| `webui_host` | _address_ | `webui_host=1.2.3.4` | The host to bind the web UI server to. Can either be `localhost` (or `127.0.0.1`) or `0.0.0.0` (available from all interfaces). By default, this is set to `localhost` to prevent access from external machines. |
| `configure_logging` | `bool` | `configure_logging=True` | If true (default), configuration of logging is allowed here. Otherwise, the user may want to configure it separately. |
| `logging_level` | _Flag_ | `logging_level=logging.INFO` | The logging level, defaults to `logging.INFO`. Ignored unless "configure_logging" is true. |
| `logging_format` | `str` | `logging_format='...'` | The logging format to use, defaults to a string containing a timestamp, filename, line number, and message. See the Ray source code `ray_constants.py` for details. Ignored unless "configure_logging" is true. |
| `temp_dir` | `str` | `temp_dir=/tmp/myray` | If provided, specifies the root temporary directory for the Ray process. Defaults to an OS-specific conventional location, e.g., `/tmp/ray`. |

See also the documentation for [ray.shutdown()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.shutdown), which is needed in some contexts.

## ray.is_initialized()

Is Ray [initialized](https://ray.readthedocs.io/en/latest/package-ref.html#ray.is_initialized)?

In [7]:
ray.is_initialized()

True

## @ray.remote()

We've used [@ray.remote](https://ray.readthedocs.io/en/latest/package-ref.html#ray.remote) a lot. You can pass arguments when using it. Here are some of them.

| Name | Type | Example | Description |
| :--- | :--- | :------ | :---------- |
| `num_cpus` | `int` | `num_cpus=4` | The number of CPU cores to reserve for this task or for the lifetime of the actor. |
| `num_gpus` | `int` | `num_gpus=1` | The number of GPU cores to reserve for this task or for the lifetime of the actor. |
| `num_return_vals` | `int` | `num_return_vals=2` | (Only for tasks, not actors.) The number of object IDs returned by the remote function invocation. |

Here's an example with and without `num_return_vals`:

In [14]:
@ray.remote(num_return_vals=3)
def tuple3(one, two, three):
    return (one, two, three)

x_id, y_id, z_id = tuple3.remote("a", 1, 2.2)
x, y, z = ray.get([x_id, y_id, z_id])
print(f'({x}, {y}, {z})')

@ray.remote
def tuple3(one, two, three):
    return (one, two, three)

xyz_id = tuple3.remote("a", 1, 2.2)
x, y, z = ray.get(xyz_id)
print(f'({x}, {y}, {z})')

(a, 1, 2.2)
(a, 1, 2.2)


### @ray.method()

Related to `@ray.remote()`, [@ray.method()](https://ray.readthedocs.io/en/latest/package-ref.html#ray.method) allows you to specify the number of return values for a method in an actor, by passing the `num_return_vals` keyword argument. None of the other `@ray.remote()` keyword arguments are allowed. Here is an example:

In [17]:
@ray.remote
class Tupleator:
    @ray.method(num_return_vals=3)
    def tuple3(self, one, two, three):
        return (one, two, three)
    
tupleator = Tupleator.remote()
x_id, y_id, z_id = tupleator.tuple3.remote("a", 1, 2.2)
x, y, z = ray.get([x_id, y_id, z_id])
print(f'({x}, {y}, {z})')   

(a, 1, 2.2)


## ray.put()

We used [`ray.get`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.gett) a lot to retrieve objects and we used actor methods to retrieve state from an actor. You can actually put objects into the object store explicitly with [`ray.put`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.put), as shown in the following example:

In [20]:
f'http://{ray.get_webui_url()}'

'http://localhost:8265'

In [7]:
id = ray.put("Hello World!")
print(f'Object returned: {ray.get(id)}')

Object returned: Hello World!


There is an optional flag you can pass `weakref=True` (defaults to `False`). If true, Ray is allowed to evict the object while a reference to the returned ID still exists. This is useful if you are putting a lot of objects into the object store and many of them might not be needed in the future. It allows Ray to more aggressively reclaim memory.

## Fetching Information

Many methods return information:

| Method | Brief Description |
| :----- | :---------------- |
| [`ray.get_gpu_ids()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get_gpu_ids) | GPUs |
| [`ray.get_resource_ids()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get_resource_ids) | Resources available to the _worker_ |
| [`ray.get_webui_url()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get_webui_url) | Ray Dashboard URL |
| [`ray.nodes()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.nodes) | Cluster nodes |
| [`ray.objects()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.objects) | Objects currently in the Object Store |
| [`ray.cluster_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.cluster_resources) | All the available resources, used or not |
| [`ray.available_resources()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.available_resources) | Resources not in use |
| [`ray.errors()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.errors) | What errrors have occurred for this job (pass `all_jobs=True` for all errors) |

In [9]:
print(f"""
ray.get_gpu_ids():          {ray.get_gpu_ids()}
ray.get_resource_ids():     {ray.get_resource_ids()}
ray.get_webui_url():        {ray.get_webui_url()}
ray.nodes():                {ray.nodes()}
ray.objects():              {ray.objects()}
ray.cluster_resources():    {ray.cluster_resources()}
ray.available_resources():  {ray.available_resources()}
ray.errors():               {ray.errors(all_jobs=True)}
""")


ray.get_gpu_ids():          []
ray.get_resource_ids():     {}
ray.get_webui_url():        localhost:8266
ray.nodes():                [{'NodeID': '4205e012427c24e7bb8dae1a70d20d9180ba85a4', 'Alive': True, 'NodeManagerAddress': '192.168.1.149', 'NodeManagerHostname': 'DWAnyscaleMBP.local', 'NodeManagerPort': 56208, 'ObjectManagerPort': 56239, 'ObjectStoreSocketName': '/tmp/ray/session_2020-04-11_17-48-23_781510_92557/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2020-04-11_17-48-23_781510_92557/sockets/raylet', 'Resources': {'object_store_memory': 32.0, 'CPU': 8.0, 'node:192.168.1.149': 1.0, 'memory': 94.0}, 'alive': True}]
ray.objects():              {ObjectID(ffffffffffffffffffffffff0100008801000000): {'DataSize': 0, 'Manager': b'B\x05\xe0\x12B|$\xe7\xbb\x8d\xae\x1ap\xd2\r\x91\x80\xba\x85\xa4'}}
ray.cluster_resources():    {'object_store_memory': 32.0, 'CPU': 8.0, 'node:192.168.1.149': 1.0, 'memory': 94.0}
ray.available_resources():  {'memory': 94.0, 'node:192.168.1.149

Recall that we used `ray.nodes()[0]['Resources']['CPU']` in the second lesson to determine the number of CPU cores on our machines:

In [10]:
import json
ray.nodes()[0]['Resources']['CPU']

8.0

## ray.timeline()

Sometimes you need to find task bottlenecks. [`ray.timeline()`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.timeline) helps. It returns a list of profiling events that can viewed as a timeline. To use the results, the easiest method is to dump the data to a JSON file by passing in `filename=...` argument. Or, you can call `json.dump(filename)` on the returned object. In either case, open chrome://tracing in a Chrome browser window (only Chrome works) and load the dumped file. 

## ray.kill(actor)

It's rarely needed, but to terminate an actor, use [`ray.kill(actor)`](https://ray.readthedocs.io/en/latest/package-ref.html#ray.kill).
It will interrupt any running tasks on the actor, causing them to fail immediately. All pending messages for the actor will be lost, as well as the actor's state. The enclosing worker also exits, althrough Ray will create a new one when needed. Any [`atexit`](https://docs.python.org/3/library/atexit.html) handlers installed in the actor's worker process will still be run. Hence, avoid using this method.